## Hands-on example: Working with data from a movie database

In this hands-on exercise, we will load a large dataset (film metadata, modified from https://www.kaggle.com/rounakbanik/the-movies-dataset#movies_metadata.csv), extract and 'clean' parts of it, perform some analysis, and plot the results.

Have fun! :)

In [ ]:
# We'll need pandas and Matplotlib
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# We'll also use 'webbrowser' to load a website (IMDb.com)
import webbrowser

In [ ]:
# Load the csv file (you may have to adjust the path depending on where this file is located on your computer!)
df = pd.read_csv('data\movies_dataset.csv')

# df.head() will show the top of the table.
# df.head()
# Show a random sample of rows in this dataset with sample(). Use the slider to see the full width of the table.
df.sample(5)

In [ ]:
# What is the size of this dataset?
print('Size (number of rows = number of films): ', len(df))

In [ ]:
# Explore
print('Synopsis of the movie stored in the first row:\n', df['overview'][0], '\n')
print('Original title of the movie in row 5436:\n', df.loc[5436]['original_title'], '\n')
print('Production companies for the movie in row 25539:\n', df.loc[25539]['production_companies'], '\n')

In [ ]:
# Filter for a given title
df[df.title=='12 Angry Men']

In [ ]:
# Load this movie (1957 version -> row 1161) in IMDb with the imdb_id.
webbrowser.open_new_tab('https://www.imdb.com/title/' + df.loc[1161, 'imdb_id'])

In [ ]:
# For the following analysis, we are interested in when movies were released, their runtimes and which languages were
# used. So we can drop all the columns we don't need, but we'll keep titles and imdb_ids.
df = df.drop(columns=['genres', 'budget', 'overview', 'production_countries', 'production_companies', 'revenue', 
                      'spoken_languages', 'vote_count', 'vote_average'])
df.head()

In [ ]:
# Now let's do some analysis with this dataset! 
# First, we'll ensure that the columns containing release dates and runtimes are formatted consistently.

# Remove rows where release_date, runtime, title, original_language are missing.
df.dropna(subset=['release_date', 'runtime', 'title', 'original_language'], inplace=True)

# Delete rows of release_date entries that are too short or don't contain '/' (badly formatted)
drop_condition = (df.release_date.str.len() < 8) | ~df.release_date.str.contains('/') 
df.drop(df[drop_condition].index, inplace=True)

# Use only the year in the release_date column (as an integer)
df['release_date'] = df['release_date'].apply(lambda x: int(x[-4:]))

In [ ]:
# Let's look at the data again. Do you see the difference in the release_date column?
df.head()

In [ ]:
# Now, let's look at the film lengths in minutes (label 'runtime'). They should be ints and larger than 0.
df.drop(df[df.runtime < 1].index, inplace=True)
df['runtime'] = df['runtime'].apply(lambda x: int(x))
           
# And language codes must be exactly two characters
df.drop(df[df.original_language.str.len()!=2].index, inplace=True)

# Finally drop rows with empty titles
df.drop(df[df.title.str.len()==0].index, inplace=True)

# Note: In pandas, the syntax df['col_name'] and df.col_name are equivalent. Choose what you like best!

In [ ]:
# Rename two columns (release_date -> year, original_language -> language)
new_names = {'release_date': 'year',
             'original_language': 'language'}
df.rename(columns=new_names, inplace=True)

# Let's look at the data again
df.sample(3)

In [ ]:
# Quick statistics for the numeric columns
df.describe()

In [ ]:
runtimes = df['runtime']
print(runtimes.shape)

years = df['year']
print(years.shape)
type(years)

In [ ]:
# What are the mean, median and maximum runtimes?
print('Mean: ', runtimes.mean())
print('Median: ', runtimes.median())
print('Longest runtime: ', runtimes.max())

In [ ]:
# Longest runtime 1256 minutes? What movie is that?
print(df[df.runtime==1256].original_title)
row_index = df.loc[df.runtime==1256].index[0]

webbrowser.open_new_tab('https://www.imdb.com/title/' + df.loc[row_index, 'imdb_id'])

In [ ]:
# Actually, it's a series, not a single movie... Let's cut this row out
df.drop(row_index, inplace=True)

In [ ]:
# Show a histogram of the runtimes
df['runtime'].hist(bins=40, range=(0, 200))

In [ ]:
# Quick plot of runtimes over years
plt.scatter(years, runtimes)
# plt.show()

In [ ]:
# As you can see, plotting all datapoints is not very useful!

# So let's group by year and plot the average runtime in each year
avg_runtimes_vs_years = df.groupby('year')['runtime'].mean()

In [ ]:
# Matplotlib can deal with the output (Series)
plt.plot(avg_runtimes_vs_years)
plt.show()

In [ ]:
# And the following syntax also works because pandas uses Matplotlib under the hood when you call .plot()
avg_runtimes_vs_years.plot()

In [ ]:
# Look at a specific language: Hindi, language code 'hi'
df_hindi_subset = df[df.language=='hi']
df_hindi_subset.sample(10)

In [ ]:
avg_runtimes_vs_years_hi = df_hindi_subset.groupby('year')['runtime'].mean()

# Plot data for all movies and for Hindi-only movies
plt.plot(avg_runtimes_vs_years, label='all movies')
plt.plot(avg_runtimes_vs_years_hi, label='Hindi language')
plt.legend(loc='lower right')
plt.title('Variation of movie runtimes over time')
plt.xlabel('year')
plt.ylabel('runtime in min (averaged per year)')
plt.show()

In [ ]:
# What about the lengths of movie titles over the years?
# Create a new column containing title lengths
df['title_length'] = df['original_title'].apply(lambda x: len(x))
df.sample(5)

In [ ]:
titlelength_vs_years = df.groupby('year')['title_length'].mean()
# We can extract the data for processing in NumPy
# title_lengths = titlelength_vs_years.to_numpy()
# ...

In [ ]:
titlelength_vs_years.plot()
plt.title('Variation of film title lengths over time')
plt.xlabel('year')
plt.ylabel('title length (# of characters)')
plt.show()

In [ ]:
# And you can explore much more... :)